In [1]:
import sys
sys.executable
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import csv
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import threading
np.random.seed(7) 
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Activation, Lambda
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.advanced_activations import ELU
from keras.regularizers import l2, activity_l2
from keras.optimizers import Adam
from keras.utils import np_utils
import math
import random

# Constants
BATCH_SIZE = 32
LEARNING_RATE = 0.00001
SAMPLES_PER_EPOCH=((20000//BATCH_SIZE)*BATCH_SIZE)
NO_OF_EPOCHS = 8

Using TensorFlow backend.


In [2]:
#Data Exploration
#Histogram visualisation to see which steering angle ranges are most overrepresented

def visualise(steering_angles):
    binwidth = 0.025
    plt.hist(steering_angles, bins=np.arange(min(steering_angles), max(steering_angles) + binwidth, binwidth))
    plt.title('Number of augmented images per steering angle')
    plt.xlabel('Steering Angle')
    plt.ylabel('# Augmented Images')
    plt.show()
    
def visualise_augmentation(image_frames, steering_angles):
    """
    Utility method to visualise augmentation operation on a random data point
    """
    X=[]
    Y=[]
    choice = np.random.randint(len(image_frames))
    x, y = load_data(image_frames[choice], steering_angles[choice])
    X.append(x)
    Y.append('Original | Steering Angle : ' + str(y))
        
    aug_bright_x = augment_brightness(x)
    X.append(aug_bright_x)
    Y.append('Brightness Augmentation| Steering Angle : '+str(y))
        
    aug_shadow_x = random_shadow(x)
    X.append(aug_shadow_x)
    Y.append('Shadow Augmentation| Steering Angle : '+str(y))
        
    hv_x,hv_y = horizontal_vertical_shift(x, y)
    X.append(hv_x)
    Y.append('H_V_Shift|Steering Angle : '+str(hv_y))
        
    flip_x, flip_y = flip(x,y)
    X.append(flip_x)
    Y.append('Flip Aug|Steering Angle : '+str(flip_y))
    
    aug_x, aug_y = augment_training_data(image_frames[choice],steering_angles[choice] )
    X.append(aug_x)
    Y.append('Combined Augmentation | Steering Angle : ' + str(aug_y))
    
    visualise_images(X,Y)
    
     
def visualise_images(X,y):
    fig, axes = plt.subplots(2,3, figsize=(10, 10))
    fig.subplots_adjust(hspace = .2, wspace=.4)
    axes = axes.ravel()
    for i in np.arange(6):
        img = X[i]
        axes[i].axis('off')
        axes[i].set_title(str(y[i]))
        axes[i].imshow(img)
    plt.show()

In [3]:
### Data Augmentation Methods ###
def augment_brightness(image):
    """
    Returns an augmented image with random darkening
    """
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    hsv = np.array(hsv, dtype = np.float64)
    # Randomly generate the brightness reduction factor
    # Add a constant so that it prevents the image from being completely dark
    random_bright = .25+np.random.uniform()
    
    # Apply the brightness reduction to the V channel
    hsv[:,:,2] = hsv[:,:,2]*random_bright
    hsv[:,:,2][hsv[:,:,2]>255]  = 255
    hsv = np.array(hsv, dtype = np.uint8)
    
    #Convert back to RGB colorspace
    new_img = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)

    return new_img 


def random_shadow(image):
    """
    Randomly distorts an image by adding a random shadow
    """
    top_y = 320*np.random.uniform()
    top_x = 0
    bot_x = 160
    bot_y = 320*np.random.uniform()
    image_hls = cv2.cvtColor(image,cv2.COLOR_RGB2HLS)
    shadow_mask = 0*image_hls[:,:,1]
    X_m = np.mgrid[0:image.shape[0],0:image.shape[1]][0]
    Y_m = np.mgrid[0:image.shape[0],0:image.shape[1]][1]
    shadow_mask[((X_m-top_x)*(bot_y-top_y) -(bot_x - top_x)*(Y_m-top_y) >=0)]=1
    if np.random.randint(2)==1:
        random_bright = .5
        cond1 = shadow_mask==1
        cond0 = shadow_mask==0
        if np.random.randint(2)==1:
            image_hls[:,:,1][cond1] = image_hls[:,:,1][cond1]*random_bright
        else:
            image_hls[:,:,1][cond0] = image_hls[:,:,1][cond0]*random_bright    
    image = cv2.cvtColor(image_hls,cv2.COLOR_HLS2RGB)
    return image


def horizontal_vertical_shift(img, steering_angle):
    """
    Returns a randomly shifted image in horizontal/ vertical direction and adjusted steering angle according to the shift
    """
     # Translation
    rows, cols, channels = img.shape
    # Translation
    tx = 100*np.random.uniform() - 100/2
    steering_angle = steering_angle + tx/100*2*.2
    
    ty = 40*np.random.uniform() - 40/2
    
    transform_matrix = np.float32([[1, 0, tx],
                                   [0, 1, ty]])
    
    translated_image = cv2.warpAffine(img, transform_matrix, (cols, rows))
    return translated_image, steering_angle
        

def flip(image, measurement):
    """
    Returns a horizontly flipped image and adjusted steering angle
    """
    new_image = cv2.flip(image,1)
    new_measurement = measurement*(-1)
    return new_image, new_measurement


def crop(image):
    """
    Returns a cropped image cropped on top and bottom to remove sky and car hood position and resized to model input size 
    """
    cropped = cv2.resize(image[60:140,:], (64,64))

    return cropped

def augment_training_data(image_frame, steering_angle):
    """
    Augment the training data with random augmentation methods
    """
    image, adjusted_steering_angle = load_data(image_frame, steering_angle)
    augmented_image = augment_brightness(image)
    #augmented_image = random_shadow(augmented_image)
    augmented_image, adjusted_steering_angle = horizontal_vertical_shift(augmented_image, adjusted_steering_angle)
    flip_image = np.random.randint(2)
    if flip_image==0:
        augmented_image, adjusted_steering_angle = flip(augmented_image, adjusted_steering_angle)
    return augmented_image, adjusted_steering_angle
    

def load_data(image_frame, steering_angle):
    """
    Randomly picks one of the [center, left, right] images in image frame and returns it along with adjusted steering angle
    """
    index = np.random.randint(3)
    if (index == 1):
        steering_angle += 0.25    # Steering angle adjustment for left camera
    elif (index == 2):
        steering_angle -= 0.25    # Steering angle adjustment for right camera
    
    image_path = image_frame[index].split('/')[-1]
    current_image_path = './data/IMG/'+image_path
    image = mpimg.imread(current_image_path)

    return image, steering_angle

In [4]:
### Data Generator###

# Adding to avoid threading issues with generators as mentioned here
#https://github.com/fchollet/keras/issues/1638
class threadsafe_iter:
    """Takes an iterator/generator and makes it thread-safe by
    serializing call to the `next` method of given iterator/generator.
    """
    def __init__(self, it):
        self.it = it
        self.lock = threading.Lock()

    def __iter__(self):
        return self
    
    def __next__(self):
        with self.lock:
            return self.it.__next__()
        
def threadsafe_generator(f):
    """A decorator that takes a generator function and makes it thread-safe.
    """
    def g(*a, **kw):
        return threadsafe_iter(f(*a, **kw))
    return g


generated_steering_angles = []
@threadsafe_generator
def generator(image_frames, steering_angles, batch_size=BATCH_SIZE):
    """
    Returns a batch of generated data by randomly applying brightness augmentation, shadow augmentation, vertical flipping, horizontal and vertical shift methods on original data. 
    """
    X_train = np.zeros((batch_size, 64, 64, 3), dtype = np.float32)
    y_train = np.zeros((batch_size,), dtype = np.float32)

    while True:
        
        image_frames, steering_angles = shuffle(image_frames, steering_angles)
        for i in range(batch_size):
            choice = np.random.randint(len(image_frames))
            x, y = augment_training_data(image_frames[choice],steering_angles[choice])
            X_train[i] = crop(x)
            y_train[i] = y
            generated_steering_angles.append(y)    
        yield X_train, y_train

In [5]:
###  Model Structure ####
def get_model():
    #Based on Comma.ai model
    model = Sequential()
    
    #Input Normalisation Layer
    model.add(Lambda(lambda x: x/127.5 - 1.,input_shape=(64,64,3)))
    
    #Convolutional Layer
    model.add(Convolution2D(16, 8, 8, subsample=(4, 4), border_mode="same", activation='elu', name='Conv1'))
    model.add(Convolution2D(32, 5, 5, subsample=(2, 2), border_mode="same", activation='elu', name='Conv2'))
    model.add(Convolution2D(64, 5, 5, subsample=(2, 2), border_mode="same", activation='elu', name='Conv3'))
    
    #Fl
    model.add(Flatten())
    model.add(Dropout(.2))
    model.add(ELU())
    model.add(Dense(512, activation='elu', name='FC1'))
    model.add(Dropout(.5))
    model.add(ELU())
    model.add(Dense(1, name='output'))

    adam = Adam(lr = 0.0001)
    model.compile(optimizer= adam, loss='mse', metrics=['accuracy'])
    
    return model

In [6]:
def start():
    
    # Load dataset
    colnames = ['center', 'left', 'right', 'steering', 'throttle', 'brake', 'speed']
    data = pd.read_csv('./data/driving_log.csv', skiprows=[0], names=colnames)

    
    ## Data Exploration ##
    
    # Get a summary of the dataset
    print(data.describe(), "\n")

    image_frames = data[['center', 'left', 'right']].values
    steering_angles = data['steering'].values

    print("Image Frames Shape:", image_frames.shape, "\n")
    print("Steering Angles Shape:", steering_angles.shape, "\n")
    print("Data loaded...")

    #Visualise steering angles spread
    #visualise(steering_angles)
    
    # Visualise image augmentation methods
    #visualise_augmentation(image_frames, steering_angles)
    
    
    # Create train and test split with 10% of the data as test data    
    image_frames, steering_angles = shuffle(np.array(image_frames), np.array(steering_angles))
    X_train, X_valid, y_train, y_valid = train_test_split(image_frames, steering_angles, test_size=0.1)


    # Initialise training and validation generators
    train_generator = generator(X_train, y_train, batch_size=32)
    validation_generator = generator(X_valid, y_valid, batch_size=32)
    
    #X, y = train_generator.__next__()
    #visualise_images(X, y)
  
    model = get_model()
    model.summary()
    model.fit_generator(train_generator, samples_per_epoch = SAMPLES_PER_EPOCH, nb_epoch=NO_OF_EPOCHS, validation_data = validation_generator, nb_val_samples = len(X_valid),verbose=1)
    model.save('model.h5')
    
    #Visualise generated data spread
    #visualise(generated_steering_angles) 
    
    print('Successfully completed training!')

In [7]:
start()

          steering     throttle        brake        speed
count  8036.000000  8036.000000  8036.000000  8036.000000
mean      0.004070     0.869660     0.001970    28.169839
std       0.128840     0.301326     0.036565     6.149327
min      -0.942695     0.000000     0.000000     0.502490
25%       0.000000     0.985533     0.000000    30.183093
50%       0.000000     0.985533     0.000000    30.186400
75%       0.000000     0.985533     0.000000    30.186640
max       1.000000     0.985533     1.000000    30.709360 

Image Frames Shape: (8036, 3) 

Steering Angles Shape: (8036,) 

Data loaded...
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 64, 64, 3)     0           lambda_input_1[0][0]             
_____________________________________________________________________________________________